In [1]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import ccxt

import src
cloudpickle.register_pickle_by_value(src) # for model portability

In [2]:
import time
import requests

base_url = 'https://api.dydx.exchange'

class Fetcher:
    def __init__(self, symbol=None):
        self.symbol = symbol
        self.keys = {
            'symbol': symbol
        }
        self.data_id = 'dydx_ohlcv'
        
    def fetch(self, last_timestamp=None):
        limit = 100
    
        if last_timestamp is None:
            start_time = self._find_start_time()
        else:
            start_time = last_timestamp + 60 * 60
            
        end_time = start_time + limit * 3600
        start_time = pd.to_datetime(start_time - 1, unit='s', utc=True).isoformat()
        end_time = pd.to_datetime(end_time, unit='s', utc=True).isoformat()
    
        url = '{}/v3/candles/{}?resolution=1HOUR&fromISO={}&toISO={}&limit={}'.format(
            base_url,
            self.symbol,
            start_time.replace('+', '%2b'),
            end_time.replace('+', '%2b'),
            limit,
        )
        print('url {}'.format(url))
        
        res = requests.get(url).json()
        # print(res)
        
        df = pd.DataFrame(res['candles'])
        df = df.rename(columns={
            'startedAt': 'timestamp',
            'open': 'op',
            'high': 'hi',
            'low': 'lo',
            'close': 'cl',
            'baseTokenVolume': 'volume', 
            'usdVolume': 'amount', 
            'startingOpenInterest': 'open_interest',
        })
        df = df.drop(columns=['updatedAt', 'market', 'resolution'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).astype(int) // 10**9
        
        for col in ['op', 'hi', 'lo', 'cl', 'volume', 'amount', 'trades', 'open_interest']:
            df[col] = df[col].astype('float')
        
        df = df.set_index('timestamp')
        df = df.sort_index()
        
        df = df.iloc[:-1] # remove partial
        
        return df
    
    def _find_start_time(self):
        limit = 100
        
        start_time = None
        end_time = int(time.time())
        
        while True:
            url = '{}/v3/candles/{}?resolution=1DAY&toISO={}&limit={}'.format(
                base_url,
                self.symbol,
                pd.to_datetime(end_time, unit='s', utc=True).isoformat().replace('+', '%2b'),
                limit,
            )
            print(url)
            
            res = requests.get(url).json()
            if len(res['candles']) == 0:
                break
                
            df = pd.DataFrame(res['candles'])
            df['timestamp'] = pd.to_datetime(df['startedAt'], utc=True)
            
            if start_time is None:
                start_time = df['timestamp'].min()
            else:
                start_time = min(start_time, df['timestamp'].min())
                
            end_time = start_time.timestamp()
                
        return start_time.timestamp()
        


In [3]:
fetcher = Fetcher(symbol='BTC-USD')
df = fetcher.fetch(last_timestamp=None)
display(df)
df = fetcher.fetch(last_timestamp=1614564000)
display(df)

https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2022-11-18T23:22:53%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2022-08-11T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2022-05-03T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2022-01-23T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2021-10-15T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2021-07-07T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2021-03-29T00:00:00%2b00:00&limit=100
https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1DAY&toISO=2021-02-25T00:00:00%2b00:00&limit=100
url https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1HOUR&fromISO=2021-02-24T23:59:59%2b00:00&toISO=2021-03-01T04:00:00%2b00:00&limit=100


,lo,hi,op,cl,volume,trades,amount,open_interest
timestamp,,,,,,,,
1614211200,50592.0,50592.0,50592.0,50592.0,0.001,1.0,50.592,0.025
1614214800,50592.0,50592.0,50592.0,50592.0,0.000,0.0,0.000,0.026
1614218400,50592.0,50592.0,50592.0,50592.0,0.000,0.0,0.000,0.026
1614222000,50592.0,50592.0,50592.0,50592.0,0.000,0.0,0.000,0.026
1614225600,50592.0,50592.0,50592.0,50592.0,0.000,0.0,0.000,0.026
...,...,...,...,...,...,...,...,...
1614549600,44512.0,44512.0,44512.0,44512.0,0.000,0.0,0.000,0.245
1614553200,44512.0,44512.0,44512.0,44512.0,0.000,0.0,0.000,0.245
1614556800,44512.0,44512.0,44512.0,44512.0,0.000,0.0,0.000,0.245


url https://api.dydx.exchange/v3/candles/BTC-USD?resolution=1HOUR&fromISO=2021-03-01T02:59:59%2b00:00&toISO=2021-03-05T07:00:00%2b00:00&limit=100


,lo,hi,op,cl,volume,trades,amount,open_interest
timestamp,,,,,,,,
1614567600,46279.0,46279.0,46279.0,46279.0,0.000,0.0,0.000,0.267
1614571200,46279.0,46279.0,46279.0,46279.0,0.000,0.0,0.000,0.267
1614574800,46536.0,46536.0,46536.0,46536.0,0.020,1.0,930.720,0.267
1614578400,46019.0,46318.0,46318.0,46019.0,0.060,2.0,2764.130,0.287
1614582000,46019.0,46019.0,46019.0,46019.0,0.000,0.0,0.000,0.287
...,...,...,...,...,...,...,...,...
1614906000,46558.0,46843.0,46843.0,46751.0,0.031,4.0,1448.158,0.000
1614909600,46473.0,46812.0,46473.0,46812.0,0.015,2.0,700.485,0.000
1614913200,47082.0,47082.0,47082.0,47082.0,0.001,1.0,47.082,0.000


In [4]:
symbols = 'BTC,ETH,LINK,ATOM,DOT,SOL,MATIC,ADA,UNI,LTC,ETC,AVAX,SUSHI'.split(',')
fetchers = []

for symbol in symbols:
    fetchers.append(Fetcher(symbol=f'{symbol}-USD'))
    
data = cloudpickle.dumps(fetchers)
data = lzma.compress(data)
with open('/home/jovyan/data/20221119_dydx_ohlcv.xz', 'wb') as f:
    f.write(data)